In [12]:
%pip install langchain_community langchain_google_genai

In [13]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


In [14]:
groq_api_key="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"



In [15]:
%pip install -qU langchain-groq


Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
# from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq


# Load environment variables from a .env file
load_dotenv()


from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")
result = llm.invoke("")
print(result.content)


Please provide me with a topic or question so I can assist you. 

For example, you could ask me:

* To write a story
* To translate a phrase
* To summarize a factual topic
* To provide information about a specific subject
* To help you brainstorm ideas


I'm ready to help! 😊 



In [17]:
import pandas as pd

file_path = ('./dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [18]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [19]:
%pip install langchain_faiss
%pip install -qU langchain-cohere

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embeddings

C:\Users\visha\AppData\Local\Temp\ipykernel_10744\151384451.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [23]:
%pip install faiss-cpu
import faiss
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))

Note: you may need to restart the kernel to use updated packages.


In [25]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [26]:
vector_store.add_documents(documents=docs)

['03d516f8-03d0-40ee-bfe4-6dcf795780af',
 'e1406b19-5e86-4980-b666-a7a2d0f86a6c',
 'e63666df-dcbf-494a-8bac-90432112f687',
 '46f76d4d-1a23-4f14-a522-53f13931a8f5',
 '8b6b735c-633d-40b3-8a91-6c86432dbf2f',
 'dad73508-2985-4113-9986-c72b07ca174c',
 '0f3b9681-c39d-48f8-8bf9-6949a0aa91ef',
 '622fe995-605e-4422-932b-5edcff291d07',
 '1d86fe98-a24a-45b4-b1ba-6e7471755e64',
 '5f6f51c1-d11c-4d3f-a67d-692372ea6100',
 '90d22fb7-21aa-4f37-82ce-fbec5c2ad8cb',
 'f1037a37-3773-438b-bb81-b8f269ded36c',
 '13f3cc6a-3665-49da-aad7-f56a39ff272d',
 '7578ce8c-d442-4bb9-9942-b57c7a72fba4',
 '447b8d99-3377-4a52-9e25-8e4772d50553',
 'ff05e58f-1cb6-45f6-b357-c34cd4bc71cf',
 'ed3eb7c7-c6fd-4462-961c-1c7605c2f7ae',
 '13a5d25a-3eb1-4750-bed8-f89eb10befd1',
 'cd6d67e3-8d9b-455b-9908-c591062b5623',
 '88ad6158-9988-409d-bb04-ceac15688f50',
 '929913d2-fe24-40e6-9d75-6e6ac398c560',
 'dbd0f886-e45c-4a58-90ef-d899eab3a17c',
 '836e1c93-fcee-4d7e-a198-215d5c280ae2',
 'cefaeee3-9e9d-4399-bedc-0b701551bf64',
 '274331cf-97bd-

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
answer= rag_chain.invoke({"input": "how can i credit in my card?"})
answer['answer']

"Please provide the context so I can help you with crediting your card. I need information about the specific card and the method of crediting.  Without context, I can't give you a helpful answer. \n"

In [ ]:
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import faiss

# Load dataset
file_path = './dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv'
data = pd.read_csv(file_path)

# Preview dataset
# print(data.head())

# Load documents
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize FAISS index
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))
docstore = InMemoryDocstore()
index_to_docstore_id = {}
# Initialize FAISS vector store
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)
# Add documents to vector store
vector_store.add_documents(documents=docs)
retriever = vector_store.as_retriever()

# Define system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Create question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)



I'm sorry, but I can't help you with crediting your card.  That would require accessing your financial information, which I am not able to do. 

You should contact your card issuer directly for assistance with crediting your card. 



In [ ]:

# Invoke chain for a sample question
answer = rag_chain.invoke({"input": "where is ?"})
print(answer['answer'])

We do ship to the {{Delivery Country}}. 

We have a wide network of delivery services that cater to different regions within the country.  

Shipping to the {{Delivery Country}} is seamless and efficient, ensuring your package arrives safely and on time. 

